In [7]:
import emoji 
import csv
import datetime
import re
from collections import Counter
import timeit
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Desktop
filepath = 'C:\\Users\\Wilm Hanke\\Documents\\GitHub\\twitter_stream\\tweet_files\\concat_all_tweets.txt'

# Mobile
#filepath = 'C:\\Users\\Wilm\\Documents\\GitHub\\twitter_stream\\tweet files\\concat_all_tweets.txt'

In [3]:
# merge all several tweet Files to one big file: 'concat_all_tweets.txt'

# uncomment for safety reasons


"""
allfiles = glob.iglob('C:\\Users\\Wilm Hanke\\Documents\\GitHub\\twitter_stream\\tweet_files\\*.txt')


for file in allfiles:
    with open(file, 'r+', encoding = 'utf-8') as f:
        with open('C:\\Users\\Wilm Hanke\\Documents\\GitHub\\twitter_stream\\tweet_files\\concat_all_tweets.txt', 'a', encoding = 'utf-8') as f2:
            for line in f:
                if not line == '\n':
                    f2.write(line)
"""

In [8]:
# creating DataFrame

df = pd.DataFrame(pd.read_csv(filepath, header = None, index_col = False))

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2775023 entries, 0 to 2775022
Data columns (total 4 columns):
0    object
1    int64
2    float64
3    object
dtypes: float64(1), int64(1), object(2)
memory usage: 84.7+ MB


In [15]:
df.columns=['id_str','in_reply_to_user_id_str','raw_text']

In [16]:
# drop tweets which startswith 'RT' --> get only original tweets, without Retweets

df_filtered_by_RT = df[~df.raw_text.str.startswith('RT')]

df_filtered_by_RT.head(1)

,id_str,in_reply_to_user_id_str,raw_text
0,1013820394304749573,NaN,People talking shit on México when their team ...


In [17]:
def extract_chars(my_str): 
    return ''.join([c for c in my_str if not(c in emoji.UNICODE_EMOJI)])

df_clean_text = df['raw_text'].apply(lambda x: extract_chars(x))

In [18]:
def extract_emojis(my_str):
    return ''.join(c for c in my_str if c in emoji.UNICODE_EMOJI)

df_extr_emojis = df['raw_text'].apply(lambda x: extract_emojis(x))

In [19]:
# unhide the strings in jupyter notebook
pd.options.display.max_rows
pd.set_option('display.max_colwidth', -1)

new_df = df_filtered_by_RT.assign(clean_text = df_clean_text, extr_emojis = df_extr_emojis)
new_df.head()

,id_str,in_reply_to_user_id_str,raw_text,clean_text,extr_emojis
0,1013820394304749573,NaN,"People talking shit on México when their team didn’t even participate in the #worldcup like sit down 😂""2018-07-02 16:23:12","People talking shit on México when their team didn’t even participate in the #worldcup like sit down ""2018-07-02 16:23:12",😂
1,1013820396154564609,NaN,We're done here#gotgotgot#WorldCup https://t.co/XJe3bzorST,We're done here#gotgotgot#WorldCup https://t.co/XJe3bzorST,
2,1013820399090454528,NaN,"I was going for Mexico, But i like Brazil too so its cool, Neymar still in it lets see if he can go all the way #WorldCup","I was going for Mexico, But i like Brazil too so its cool, Neymar still in it lets see if he can go all the way #WorldCup",
3,1013820400155779072,NaN,Serbian President Vucic Believed in Russia's Victory Over Spain at World Cup - Ambassador https://t.co/7ffKHBwGl8… https://t.co/zakD00KJyH,Serbian President Vucic Believed in Russia's Victory Over Spain at World Cup - Ambassador https://t.co/7ffKHBwGl8… https://t.co/zakD00KJyH,
4,1013820402907267072,NaN,Bshshshshshs https://t.co/qdIiKoQViV,Bshshshshshs https://t.co/qdIiKoQViV,


In [21]:
# convert in_reply_to_user_id_str to type(int)
"""####
komplettes dataframe mit NaN in reply_id droppen
datatypen converten 
neues df assignen
####

def convert_to_int(_num):
    # einige values beginnen mit minus --> abs()
    return new_df['in_reply_to_user_id_str'].dropna().astype(np.int64).abs()

df_convert_to_int = new_df['in_reply_to_user_id_str'].apply(lambda x: convert_to_int(x))
"""
df_replies = new_df.dropna()
df_replies['in_reply_to_user_id_str'] = df_replies['in_reply_to_user_id_str'].astype(np.int64).abs()

df_replies.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,id_str,in_reply_to_user_id_str,raw_text,clean_text,extr_emojis
16,1013820416522010625,826191066478977024,@TruthMatters13 @FreedomFlagship @Greenhornet1968 @wwwillstand @gregory_pulse @wolfeatworld @BASEDinSB… https://t.co/jMBJePB5Wv,@TruthMatters13 @FreedomFlagship @Greenhornet1968 @wwwillstand @gregory_pulse @wolfeatworld @BASEDinSB… https://t.co/jMBJePB5Wv,
19,1013820419680370688,3007304426,@hotcheethoe_ Haters gonna hate but Neymar's gonna score and create ;)#BRAMEX #BRA #MEX #WorldcupRussia2018 #worldcup,@hotcheethoe_ Haters gonna hate but Neymar's gonna score and create ;)#BRAMEX #BRA #MEX #WorldcupRussia2018 #worldcup,
39,1013820436201656321,7302282,"After their 2-0 loss to Brazil, this is Mexico's 7th straight #WorldCup loss in the Round of 16.The Curse of the… https://t.co/yRXaInVhVB","After their 2-0 loss to Brazil, this is Mexico's 7th straight #WorldCup loss in the Round of 16.The Curse of the… https://t.co/yRXaInVhVB",
45,1013820447073288192,826191066478977024,@TruthMatters13 @Linking_Mercury @MrsKHuie @LindaMarie1182 @HSara1975 @Bruce80649809 @TEN4Texas @david_d1421… https://t.co/ZnrAKaWeq0,@TruthMatters13 @Linking_Mercury @MrsKHuie @LindaMarie1182 @HSara1975 @Bruce80649809 @TEN4Texas @david_d1421… https://t.co/ZnrAKaWeq0,
47,1013820450617679872,22777077,"@cardell5000 @LouDobbs @realDonaldTrump @POTUS Cardell, you have allowed the Fake News Mafia to brainwash you &amp; “Ru… https://t.co/GB8EjZOtIT","@cardell5000 @LouDobbs @realDonaldTrump @POTUS Cardell, you have allowed the Fake News Mafia to brainwash you &amp; “Ru… https://t.co/GB8EjZOtIT",


In [25]:
# rows only with emoji

new_df[new_df.extr_emojis != ''].head()

,id_str,in_reply_to_user_id_str,raw_text,clean_text,extr_emojis
0,1013820394304749573,NaN,"People talking shit on México when their team didn’t even participate in the #worldcup like sit down 😂""2018-07-02 16:23:12","People talking shit on México when their team didn’t even participate in the #worldcup like sit down ""2018-07-02 16:23:12",😂
8,1013820404601942017,NaN,"Did you hear about the hungry clock? It went back four seconds. 😄 #Gemstars #Jokes #Smile 😄""2018-07-02 16:23:15","Did you hear about the hungry clock? It went back four seconds. #Gemstars #Jokes #Smile ""2018-07-02 16:23:15",😄😄
18,1013820413732782082,NaN,"Now that's what you call a game....action, suspense, drama &amp; comedy! 🇧🇷⚽🏆#BRAMEX #WorldCup #soccer… https://t.co/16nhJna0Yz","Now that's what you call a game....action, suspense, drama &amp; comedy! 🇧🇷#BRAMEX #WorldCup #soccer… https://t.co/16nhJna0Yz",⚽🏆
24,1013820425028194307,NaN,"Listening to the match, think it’s Brazil’s to loose... maybe. It’d b class if #Firminho done a ‘no look’ lifting of the cup 😵🏆#WorldCup2018","Listening to the match, think it’s Brazil’s to loose... maybe. It’d b class if #Firminho done a ‘no look’ lifting of the cup #WorldCup2018",😵🏆
38,1013820438043152384,NaN,Well done Brazil! But Neymar be trippin’ 😂😂😂 #neymar #brazil #fifaworldcup #footballmemes #southafricanblogger https://t.co/8M6BfQ5Ne8,Well done Brazil! But Neymar be trippin’ #neymar #brazil #fifaworldcup #footballmemes #southafricanblogger https://t.co/8M6BfQ5Ne8,😂😂😂


In [23]:
# unhide the strings in jupyter notebook
pd.options.display.max_rows
pd.set_option('display.max_colwidth', -1)


#new_df.clean_text[new_df['clean_text']==997055707471003653]
type(df.iloc[15]['in_reply_to_user_id_str'])

numpy.float64

In [ ]:
# nehme eine eine id_str und suche, ob es eine in_reply_to_user_id_str gibt, 
# wenn ja, hänge kompletten tweet (id_str, in_reply_to_user_id_str, raw_text, clean_text, extr_emojis) 
# rechts neben den initial_tweet

def extract_reply(my_str):
    if df.loc[df['in_reply_to_user_id_str'] == df['id_str']]
        return 

df_reply_tweets = df['raw_text'].apply(lambda x: extract_emojis(x))